<a href="https://colab.research.google.com/github/xhiroga/til/blob/main/software-engineering/haystack/quickstart/heystack_QuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Build a Scalable Question Answering System | Haystack](https://haystack.deepset.ai/tutorials/03_scalable_qa_system)

In [1]:
%%bash

nvidia-smi

Sun Sep 24 02:33:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for farm-haystack[colab,inference] from https://files.pythonhosted.org/packages/31/db/e81141e15cecf1abc1238d9aee55f66310274878a034dd96603703620e9c/farm_haystack-1.20.1-py3-none-any.whl.metadata
  Obtaining dependency information for canals==0.7.0 from https://files.pythonhosted.org/packages/08/2f/62e9e455c4cf183ebf6672c47befabb6a36a37a8cdc86256a5632d6ace10/canals-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for events from https://files.pythonhosted.org/packages/25/ed/e47dec0626edd468c84c04d97769e7ab4ea6457b7f54dcb3f72b17fcd876/Events-0.5-py3-none-any.whl.metadata
  Obtaining dependency information for posthog from https://files.pythonhosted.org/packages/a7/73/35758818228c70348be4c3c66a76653c62e894e0e3c3461453c5

In [4]:
from haystack.telemetry import tutorial_running

tutorial_running(1)

In [5]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [6]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [7]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/build_your_first_question_answering_system"

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip",
    output_dir=doc_dir,
)

INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip to 'data/build_your_first_question_answering_system'


True

In [8]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Updating BM25 representation...: 100%|██████████| 2359/2359 [00:00<00:00, 14455.28 docs/s]


{'documents': [<Document: {'content': "\n\n'''Melisandre of Asshai''' is a fictional character in the ''A Song of Ice and Fire'' series of fantasy novels by American author George R. R. Martin and its television adaptation ''Game of Thrones''.  She is a priestess of the god R'hllor (also called the Red God or the Lord of Light) from the continent Essos and a close advisor to King Stannis Baratheon in his campaign to take the Iron Throne. She is often nicknamed '''the Red Woman''' due to the color of her hair and clothes, and has mysterious powers over fire and shadow. She is a prominent example of Martin's use of magic within the story, and is the source of several important prophecies that guide the narrative.\n\nIntroduced in ''A Clash of Kings'' (1998), Melisandre has come to Westeros to propagate her faith in the Red God.  She subsequently appeared in Martin's ''A Storm of Swords'' (2000) and ''A Dance with Dragons'' (2011).  Melisandre is not a point-of-view character in the first

In [9]:
!ls data/build_your_first_question_answering_system

0_Game_of_Thrones__season_8_.txt
101_Titties_and_Dragons.txt
102_The_Princess_and_the_Queen.txt
10_Beyond_the_Wall__Game_of_Thrones_.txt
118_Dark_Wings__Dark_Words.txt
119_Walk_of_Punishment.txt
11_The_Dragon_and_the_Wolf.txt
120_And_Now_His_Watch_Is_Ended.txt
121_The_Bear_and_the_Maiden_Fair.txt
126_Kissed_by_Fire.txt
127_The_Climb__Game_of_Thrones_.txt
129_Second_Sons.txt
12_Fire.txt
130_Game_of_Thrones_title_sequence.txt
131_Mhysa.txt
133_Game_of_Thrones__Season_5__soundtrack_.txt
134_Game_of_Thrones__Season_6__soundtrack_.txt
135_Game_of_Thrones__Season_7__soundtrack_.txt
136_Game_of_Thrones__Season_8__soundtrack_.txt
145_Elio_M._García_Jr._and_Linda_Antonsson.txt
146_The_Sons_of_the_Dragon.txt
148_Game_of_Thrones__Winter_Is_Coming.txt
151_Ellaria_Sand.txt
154_Margaery_Tyrell.txt
160_Viserys_Targaryen.txt
191_Gendry.txt
193_Lord_Snow.txt
194_A_Song_of_Ice_and_Fire.txt
195_World_of_A_Song_of_Ice_and_Fire.txt
197_A_Game_of_Thrones.txt
198_A_Clash_of_Kings.txt
199_A_Storm_of_Swords.tx